In [89]:
import mbuild as mb
import os
from parmed import load_file
import re
from mbuild.lib.atoms import H
import numpy as np
import random
import pubchempy as pcp
from rdkit import Chem 
from  Main_functions import *

The following example is based on the work carried out by Chehrazi 
https://pubs.acs.org/doi/10.1021/acsomega.3c10108

<img src="../images/TETA_TMC_image.png">


First step the main amine, TETA (triethylenetetramine) is fetched by name from the PubChem database.

In [90]:
Polymer_Compound = mb.load("../files/TETA_monomer.mol2")


/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


In [91]:
Polymer_Compound.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now the monomer has to be modified by adding the ports.
The loaded structure has a hydrogen at each end of the TETA monomer replaced by a F atom in order to streamline the process.  
The next step is adding the ports for the bonding of monomers with TMC to take place.

In [92]:
ports_to_add_Dummy_atoms_Polymer_Compound = {

     "NH2C" : [
        ["N", # The first atom is the "anchor" or base atom 
           "H", "F", "C"], # Bonded atoms to Carbon 
        ["F"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],
}

In [93]:
compound_1_port_PC = Add_port_CR(Polymer_Compound,
                                type = "NH2C",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_Polymer_Compound)    
    
print("Added ports to the molecule")
tmp_cmp_PC = mb.clone(compound_1_port_PC)

#Adding second port to the repeat unit        
compound_with_2_ports_PC = Add_port_CR(tmp_cmp_PC,
                                    type = "NH2C",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_Polymer_Compound)      
print("Added 2 ports to the molecule")

Added ports to the molecule


Added 2 ports to the molecule


In [94]:
compound_with_2_ports_PC.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

After adding the ports to the polymer monomer, now the Trimesoyl Chloride (TMC) is loaded and the ports are added.
The TMC molecule has two fluorine that replaced the chloride in order to streamline the process.

In [95]:
tmc_c  = mb.load("../files/TMC_1.mol2")

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


In [96]:
tmc_c.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [97]:
ports_to_add_Dummy_atoms_TMC = {

     "C_Cl" : [
        ["C", # The first atom is the "anchor" or base atom 
        "C", "O","F"], # Bonded atoms to Carbon 
        ["F"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],
}

In [98]:
compound_1_port_TMC = Add_port_CR(tmc_c,
                                type = "C_Cl",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_Dummy_atoms_TMC)    
    
print("Added ports to the molecule")
tmp_cmp_TMC = mb.clone(compound_1_port_TMC)

#Adding second port to the repeat unit        
compound_with_2_ports_TMC = Add_port_CR(tmp_cmp_TMC,
                                    type = "C_Cl",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_Dummy_atoms_TMC)      
print("Added 2 ports to the molecule")






Added ports to the molecule







Added 2 ports to the molecule


In [99]:
compound_with_2_ports_TMC.visualize(show_ports = True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Now the polymer is built with the combination of both molecules using mbuild's functionality for co-polymers

In [100]:
Single_Repeat_Unit = mb.recipes.Polymer(monomers=[compound_with_2_ports_PC, compound_with_2_ports_TMC ])
Single_Repeat_Unit.build( n=1, sequence="AB")
Single_Repeat_Unit.visualize(show_ports = True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The repeat unit has been built, now the ports will be added to the repeat unit in the NH2-C and in the C(H)(O), 
while at the same time making sure the chloride atom is replaced in the C(Cl)(O) group in TMC

In [101]:
ports_to_add_RU = {
    "NH2-C" : [
        ["N",  # The first atom is the "anchor" or base atom 
          "C", "H", "H",], # Bonded atoms to Carbon 
        ["H"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],

      "C-OH" : [
        ["C",  # The first atom is the "anchor" or base atom 
          "C", "H", "O",], # Bonded atoms to Carbon 
        ["H"], # Atom to remove and create port
        [3], # Total number of bonds in carbon atom 
        ],


}

compound_1_port_SRU = Add_port_CR(Single_Repeat_Unit,
                                type = "NH2-C",
                                name = "up",
                                manual_ports= True,
                                ports_dict = ports_to_add_RU)    
    
print("Added ports to the molecule")
tmp_cmp_SRU = mb.clone(compound_1_port_SRU)

#Adding second port to the repeat unit        
compound_with_2_ports_SRU = Add_port_CR(tmp_cmp_SRU,
                                    type = "C-OH",
                                    name = "down",
                                    manual_ports= True,
                                    ports_dict = ports_to_add_RU)      
print("Added 2 ports to the molecule")
compound_with_2_ports_SRU = Replace_Atom_with_dummy(Compound = compound_with_2_ports_SRU,
                            replacement_atom = "Cl",
                            anchor_name = "H",
                            list_atoms_bonded_anchor = ["C"],
                            neighbor_anchor_name = "C",
                            list_atoms_bonded_neighbor_anchor = ["O", "H", "C"],
                            )

Added ports to the molecule








Added 2 ports to the molecule


In [102]:
compound_with_2_ports_SRU.visualize(show_ports = True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The TETA molecule, which will serve as crosslinker.
The crosslinker will be modified by adding dummy Sulfur atoms in the NH2 group to facilitate the crosslinking

In [103]:
crosslinker_1 = mb.load("../files/TETA_monomer_ST.mol2")
crosslinker_1.visualize()

/raid6/homes/ordoriaf/miniconda3/envs/andres/lib/python3.10/site-packages/mdtraj/core/trajectory.py:441: UserWarning: top= kwargs ignored since this file parser does not support it
  warnings.warn("top= kwargs ignored since this file parser does not support it")


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [104]:
Dummy_Crosslinker_Polymer = Replace_Atom_with_dummy(Compound = crosslinker_1,
                            replacement_atom = "S",
                            anchor_name = "N",
                            list_atoms_bonded_anchor = ["H", "H", "C",],
                            neighbor_anchor_name = "C",
                            list_atoms_bonded_neighbor_anchor = ["N", "H", "H", "C"],
                            )

Found one
Found one


In [105]:
Dummy_Crosslinker_Polymer.visualize(show_ports= True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [106]:
compound_with_2_ports_SRU.visualize(show_ports=True)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

With the modified crosslinker (TETA monomer with sulfur atoms in place of Nitrogen atoms) and the single repeat unito with two ports (TETA+ TMC with two ports), the next step is calling the crosslinking function

In [107]:

RU = 5
N_chains = 5
n_sites = 1

dummy_atom_dictionary = {
                         "S": "N",}


box_system, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
    main_name = "TETA",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    Pre_existing_repeat_unit =True,
    Repeat_unit_compound_with_ports = compound_with_2_ports_SRU, # Here we pass already the repeat unit with ports

    Dummy_atoms = True,
    Dummy_atom_dictionary = dummy_atom_dictionary,

    
    repeat_units = 5,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 1.2,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                              #Crosslinker details
    crosslinker_name = "BASE",                                # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports
    Crosslinker_compound = Dummy_Crosslinker_Polymer,
    

 
                                                              #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 40,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "C",
                            "target_atom" : "CL",
                            "bonded_atoms": ["C", "CL", "O"], 
                            "anchor_atom_n_bonds" : 3,
                            },
                            
                          
                            ],
    
    
    crosslinker_backbone_target_list = [
                        {
                        "anchor_atom" : "S",
                        "target_atom" : ["H"],
                        "anchor_atom_n_bonds" : 3,
                        "extra_pruning": [],
                        "exclude": [],
                        },
                        
                    
    ],
    
    prune_extra_flag =False,                               # No pruning extra atoms 
    crosslink_sites_per_molecule = 1 ,                     # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
   
    

        
    
    
)


for suffix in ["mol2", "pdb"]:
    box_system.save(f"../output/From_File_5_3_TETA-TMC-Short.{suffix}",overwrite = True)

Number of ports per chain 2
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 26, 2: 53}
To attain the degree of crosslinking 40% 
 4 crosslinkers are needed 
 2 between each polymer chain
The actual attainable value of crosslinking is 53%
Which is the highest crosslinking percentage attainable
Repeat unit passed to function
Done with loading compounds
{'POL-0': [(0, array([ 1.19111062, -0.52304207,  0.00146747])), (1, array([ 2.48654068, -1.95114056, -0.35928151])), (2, array([ 3.58802423, -3.45563868,  0.24978631])), (3, array([ 5.26852578, -4.46701759,  0.22201161])), (4, array([ 6.23961665, -6.17133873,  0.21799244]))], 'POL-1': [(5, array([ 1.19111062, -0.52304207,  1.20146747])), (6, array([ 2.48654068, -1.95114056,  0.84071849])), (7, array([ 3.58802423, -3.45563868,  1.44978631])), (8, array([ 5.26852578, -4.46701759,  1.42201161])), (9, array([ 6.23961665, -6.17133873,  1.41799244

In [108]:
box_system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The crosslinker can be at the same time the crosslinked polymer chain (TETA +TMC), using mBuild's copolymer recipe and the monomers previously loaded

In [109]:
repeat_units_AB_copolymer = 1
COP_polymer = mb.recipes.Polymer(monomers=[compound_with_2_ports_PC, compound_with_2_ports_TMC ])
COP_polymer.build( n=repeat_units_AB_copolymer, sequence="ABA")
COP_polymer.visualize()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

Adding the dummy atoms at the NH2 groups

In [110]:
COP_polymer_dummy_atoms = Replace_Atom_with_dummy(Compound = COP_polymer,
                            replacement_atom = "S",
                            anchor_name = "N",
                            list_atoms_bonded_anchor = ["H", "H", "C",],
                            neighbor_anchor_name = "C",
                            list_atoms_bonded_neighbor_anchor = ["N", "H", "H", "C"],
                            )

Found one
Found one


In [111]:
COP_polymer_dummy_atoms.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [112]:

RU = 5
N_chains = 5
n_sites = 1

dummy_atom_dictionary = {
                         "S": "N",}


box_system, Number_Crosslinkers_Used, Real_Degree_Crosslinking, Polymer_Residue_Name, Crosslinker_Resiude_Name = Crosslink_Pipeline(
    main_name = "TETA",                                   # Monomer to polimerize gives PVA ; not vinylic alcohol as it's insaturated
    Pre_existing_repeat_unit =True,
    Repeat_unit_compound_with_ports = compound_with_2_ports_SRU, # Here we pass already the repeat unit with ports

    Dummy_atoms = True,
    Dummy_atom_dictionary = dummy_atom_dictionary,

    
    repeat_units = 5,                                       # Repeat Units in polymer chain
    number_of_polymers = 3,                                   # Number of polymer chains inside the box
    depth_value = 5,                                          # Spacing between polymer chains in box
    Polymer_residue_name = "POL",                             # name of polymer chain residue
    Polymer_port_name = "Port",                               # name of polymer chain backbone ports

                                                              #Crosslinker details
    crosslinker_name = "BASE",                                # Name of crosslinker agent
    Crosslinker_residue_name = "CRS",                         # name of crosslinker residue name
    Crosslinker_port_name = "Site",                           # name of cr chain backbone ports
    Crosslinker_compound = COP_polymer_dummy_atoms,
    

 
                                                              #Crosslinking details
    sites_per_repeat_unit = 1,                                # Number of functional groups (f.ex hydroxyls in PVA in each chain)
    desired_CR_degree = 40,                                   # Degree in %, that percentage (f.ex number of hydroxyls attached to the crosslinker in PVA, not hydroxyls anymore)
    
    
    polymer_backbone_target_list = [
                            {
                            "anchor_atom" : "C",
                            "target_atom" : "CL",
                            "bonded_atoms": ["C", "CL", "O"], 
                            "anchor_atom_n_bonds" : 3,
                            },
                            
                          
                            ],
    
    
    crosslinker_backbone_target_list = [
                        {
                        "anchor_atom" : "S",
                        "target_atom" : ["H"],
                        "anchor_atom_n_bonds" : 3,
                        "extra_pruning": [],
                        "exclude": [],
                        },
                        
                    
    ],
    
    prune_extra_flag =False,                               # No pruning extra atoms 
    crosslink_sites_per_molecule = 1 ,                     # Sites per molecule  (forms 1 connection with each polymer chain) f.ex glutaraldehyde forms 2 per chain
        
   
    

        
    
    
)


for suffix in ["mol2", "pdb"]:
    box_system.save(f"../output/From_File_5_3_TETA-TMC-Long.{suffix}",overwrite = True)

Number of ports per chain 2
This values are the number of crosslinking monomers that will be bonded between each pair of chains to attain the desire degree of crosslinking
{1: 26, 2: 53}
To attain the degree of crosslinking 40% 
 4 crosslinkers are needed 
 2 between each polymer chain
The actual attainable value of crosslinking is 53%
Which is the highest crosslinking percentage attainable
Repeat unit passed to function
Done with loading compounds
{'POL-0': [(0, array([ 1.19111062, -0.52304207,  0.00146747])), (1, array([ 2.48654068, -1.95114056, -0.35928151])), (2, array([ 3.58802423, -3.45563868,  0.24978631])), (3, array([ 5.26852578, -4.46701759,  0.22201161])), (4, array([ 6.23961665, -6.17133873,  0.21799244]))], 'POL-1': [(5, array([ 1.19111062, -0.52304207,  5.00146747])), (6, array([ 2.48654068, -1.95114056,  4.64071849])), (7, array([ 3.58802423, -3.45563868,  5.24978631])), (8, array([ 5.26852578, -4.46701759,  5.22201161])), (9, array([ 6.23961665, -6.17133873,  5.21799244

In [113]:
box_system.visualize()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.